In [ ]:
import pickle

with open(r'D:\PythonWorkspace\gitrepo\Lux_project\model\lrg_v1.pkl', 'rb') as f:
    lrg_clf = pickle.load(f)

lrg_clf

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(r'D:\PythonWorkspace\gitrepo\Lux_project\data\player_data.csv')
df

In [ ]:
def dataset_for_me_v2(champion, lane=None, tier=None):

    tier_df = df
    
    tier = tier_df['player_lane']
    team1_df = tier_df.iloc[:, 3:10]
    team1_df = pd.concat([tier, team1_df], axis=1)
    team2_df = tier_df.iloc[:, 10:17]
    team2_df = pd.concat([tier, team2_df], axis=1)


    team2_df.rename(columns={
            "rank": "rank",
            "player_6": "player_1",
            "player_7": "player_2",
            "player_8": "player_3",
            "player_9": "player_4",
            "player_10": "player_5",
            "bdht_2": "bdht_1",
            "win_2": "win_1"
    }, inplace=True)


    fn_df = pd.concat([team1_df, team2_df], axis=0, ignore_index=True)

    dataset = pd.DataFrame(columns=[
            'rank', 'lane', 'kills', 'deaths', 'assists', 'cs', 'barons', 'dragons', 'heralds', 'towers', 'total_kills', 'total_deaths', 'win'
        ])

    for i in range(len(fn_df)):
        tier = fn_df.iloc[i, 0]
        team_df = fn_df.iloc[i, 6].split(',')
        team_total_kills = 0
        team_total_deaths = 0

        for k in range(5):
            champ_df = fn_df.iloc[i, k+1].split(',')
            team_total_kills += int(champ_df[2])
            team_total_deaths += int(champ_df[3])

        for j in range(5):
            champ_df = fn_df.iloc[i, j+1].split(',')

            if champ_df[0] == champion:
                if lane == None:
                    position = champ_df[1].strip()      # str
                else:
                    if lane == champ_df[1].strip():
                        position = champ_df[1].strip()      # str
                    else:
                        continue
                kills = champ_df[2]         # int
                deaths = champ_df[3]        # int
                assists = champ_df[4]       # int
                cs = champ_df[5]            # int
                barons = team_df[0]         # int
                dragons = team_df[1]        # int
                heralds = team_df[2]        # int
                towers = team_df[3]         # int
                win = fn_df.iloc[i, 7]      # bool

                dataset.loc[len(dataset)] = [tier, position, kills, deaths, assists, cs, barons, dragons, heralds, towers, team_total_kills, team_total_deaths, win]
    
    return dataset

dataset_for_me_v2("Lux")
# tier = 단일 / [n1, n2, ...]
# lane = None / 지정(단일)

In [ ]:
def dataset_for_me_v1(champion, lane=None, tier=None):

    tier_df = df
    
    tier = tier_df['player_lane']
    team1_df = tier_df.iloc[:, 3:10]
    team1_df = pd.concat([tier, team1_df], axis=1)
    team2_df = tier_df.iloc[:, 10:17]
    team2_df = pd.concat([tier, team2_df], axis=1)


    team2_df.rename(columns={
            "rank": "rank",
            "player_6": "player_1",
            "player_7": "player_2",
            "player_8": "player_3",
            "player_9": "player_4",
            "player_10": "player_5",
            "bdht_2": "bdht_1",
            "win_2": "win_1"
    }, inplace=True)


    fn_df = pd.concat([team1_df, team2_df], axis=0, ignore_index=True)

    dataset = pd.DataFrame(columns=[
            'rank', 'lane', 'kills', 'deaths', 'assists', 'cs', 'barons', 'dragons', 'heralds', 'towers', 'win'
        ])

    for i in range(len(fn_df)):
        tier = fn_df.iloc[i, 0]
        team_df = fn_df.iloc[i, 6].split(',')
        team_total_kills = 0
        team_total_deaths = 0

        for j in range(5):
            champ_df = fn_df.iloc[i, j+1].split(',')

            if champ_df[0] == champion:
                if lane == None:
                    position = champ_df[1].strip()      # str
                else:
                    if lane == champ_df[1].strip():
                        position = champ_df[1].strip()      # str
                    else:
                        continue
                kills = champ_df[2]         # int
                deaths = champ_df[3]        # int
                assists = champ_df[4]       # int
                cs = champ_df[5]            # int
                barons = team_df[0]         # int
                dragons = team_df[1]        # int
                heralds = team_df[2]        # int
                towers = team_df[3]         # int
                win = fn_df.iloc[i, 7]      # bool

                dataset.loc[len(dataset)] = [tier, position, kills, deaths, assists, cs, barons, dragons, heralds, towers, win]
    
    return dataset

dataset_for_me_v1("Lux")
# tier = 단일 / [n1, n2, ...]
# lane = None / 지정(단일)

In [ ]:
xy = dataset_for_me_v1("Lux", lane="MIDDLE")

x = xy.iloc[:, 2:-1]
y = xy.iloc[:, [-1]]

lrg_clf.predict(x)

In [ ]:
winrates = lrg_clf.predict_proba(x)
win = 0
for winrate in winrates:
    if winrate[1] > 0.5:
        print("[WIN] {:.2f}%".format(winrate[1]*100))
        win += 1
    else:
        print("[LOSE] {:.2f}%".format(winrate[1]*100))
print(f"Result: {win}/{len(winrates)} games win!")
print("Total winrate: {:.2f}%".format(win/len(winrates)*100))

In [ ]:
y.columns = ['result']
y_pred = pd.DataFrame(lrg_clf.predict(x), columns=['prediction'])
y_proba = [elem[1] for elem in winrates]
y_proba = pd.DataFrame(y_proba, columns=['proba'])
isright = [(y.iloc[i,0] == y_pred.iloc[i,0]) for i in range(len(y))]
isright = pd.DataFrame(isright, columns=['right'])
mcdf = pd.concat([y, y_pred, y_proba, isright], axis=1)

right = mcdf['right'].value_counts()
print(f'Right: {right[1]}/{len(mcdf)}')

mcdf
# correct = 0
# for i in range(len(y)):
#     iscorrect = (lrg_clf.predict(x)[i] == y.iloc[i, 0])
#     if iscorrect:
#         correct += 1
# print(f"{correct/len(y)}")


생각보다 잘 맞추는데... 18건 중 4건은 예측실패다. 왜 그럴까? \
50%에 가까운 승률이면 접전이여서 틀릴 수도 있지만, 거의 이겼다(>.80)고 판단한 2건은 그냥 넘어가기 어렵다고 생각한다. \
왜 그런지에 대한 추측을 달아본다.

> 1. 모델이 적합하지 않다. \
ㄴ 이건 아닌 것 같다...
> 2. 주어진 피쳐가 모자라다. 가령, 1 & 2팀의 전체성적같은 것도 필요로 하는 것 같다. 나 말고 다른 팀이 말아먹어놨을 수 있으니... \
ㄴ 실제로 데이터를 조사해보니... 다른 팀원이 심각하게 던져놨다. 상대도 개차반이지만... 팀 전체 킬수를 피쳐로 도입해보자.
> 3. 정말 우연의 일치로 틀렸다. 다 이긴 판인데도 넥서스백도 같은 것을 당해 졌거나... 등등 \
ㄴ 그러지 않았다. 이런 경우는 200판에 1~3판 정도 밖에 안된다고 생각한다. 이번 데이터에서도 없었다.

* v2로 total_kd를 도입했는데 더욱 안좋다.
* train set을 현 티어인 silver=ii에 맞추니 예측률이 더 올랐다. v1버전으로 티어에 맞춰서 사용해보자.
* v1에서는 전 티어에서 보아야 예측률이 더 높다. v2에서는 티어에 맞추어야 예측률이 더 높다.